In [3]:
import pickle
import numpy as np
import requests

In [4]:
def predict_single(customer, dv, model):
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0]

In [5]:
with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

/home/martin/anaconda3/envs/ml-zoomcamp/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

/home/martin/anaconda3/envs/ml-zoomcamp/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
def predict():
    prediction = predict_single(customer, dv, model)
    score = prediction >= 0.5

    result = {
        'score': bool(score),
        'score_probability': float(prediction),
    }

    return result

In [8]:
customer = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

In [9]:
predict()

{'score': False, 'score_probability': 0.16213414434326598}

In [10]:
'''
from flask import Flask, request, jsonify

app = Flask('score')

@app.route('/predict', methods=['POST'])
def predict():
    customer = request.get_json()

    prediction = predict_single(customer, dv, model)
    score = prediction >= 0.5

    result = {
        'score': bool(score),
        'score_probability': float(prediction),
    }

    return jsonify(result)

app.run(debug=True, host='0.0.0.0', port=9696)
'''

"\nfrom flask import Flask, request, jsonify\n\napp = Flask('score')\n\n@app.route('/predict', methods=['POST'])\ndef predict():\n    customer = request.get_json()\n\n    prediction = predict_single(customer, dv, model)\n    score = prediction >= 0.5\n\n    result = {\n        'score': bool(score),\n        'score_probability': float(prediction),\n    }\n\n    return jsonify(result)\n\napp.run(debug=True, host='0.0.0.0', port=9696)\n"

In [ ]:
'''
FROM svizor/zoomcamp-model:3.9.12-slim

ENV PYTHONUNBUFFERED=TRUE

RUN pip --no-cache-dir install pipenv

WORKDIR /app

COPY ["Pipfile", "Pipfile.lock", "./"]
RUN pipenv install --deploy --system && \
    rm -rf /root/.cache

COPY ["*.py", "model1.bin", "dv.bin", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "score_serving:app"] 

'''

In [ ]:
'''
docker build -t churn-prediction .

docker run -it -p 9696:9696 churn-prediction:latest
'''

In [11]:
customer = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}

In [12]:
url = 'http://localhost:9696/predict'
response = requests.post(url, json=customer)
result = response.json()
result

{'score': True, 'score_probability': 0.9282218018527452}